In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense


data = """The global economic landscape is navigating through a period of significant uncertainty and transformation. Inflationary pressures continue to challenge policymakers, with central banks around the world grappling with the need to raise interest rates to combat rising prices, while also trying to avoid stifling economic recovery. The aftermath of the COVID-19 pandemic is still being felt, as disruptions to global supply chains and labor markets persist, leading to bottlenecks in production and distribution.

Geopolitical tensions, particularly in regions like Eastern Europe and Asia, have exacerbated energy crises, contributing to volatile fuel prices and impacting global trade. Additionally, the shift towards digitalization and green energy is accelerating, driven by both necessity and innovation. However, this transition is creating disparities, as industries and workers in traditional sectors face the challenge of adapting to new technologies and regulatory environments.

Moreover, emerging markets are experiencing mixed fortunes; while some are benefiting from higher commodity prices, others are struggling with debt burdens and capital outflows. The rising cost of living is fueling social unrest in several parts of the world, highlighting the growing inequality and the need for more inclusive economic policies.

Amid these challenges, there are also opportunities for growth and resilience. Investments in renewable energy, technology, and infrastructure are paving the way for more sustainable economic models. Governments and businesses are increasingly focusing on building supply chain resilience, fostering innovation, and promoting digital inclusion. While the road ahead is uncertain, the global economy is gradually adapting to these new realities, with a focus on sustainable and inclusive growth as key drivers for future stability."""


# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
total_words = len(tokenizer.word_index) + 1

# Convert text to sequences of words
input_sequences = []
for line in data.split(". "):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
X = input_sequences[:,:-1]
y = input_sequences[:,-1]

# One-hot encode the labels
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

# Step 3: Build Model
model = Sequential()
model.add(Embedding(total_words, 10, input_length=max_sequence_len-1))
model.add(SimpleRNN(100))
model.add(Dense(total_words, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.summary()

# Train the model
model.fit(X, y, epochs=100, verbose=1)

# Step 4: Generate Text
def predict_next_word(model, tokenizer, text, max_sequence_len):
    for _ in range(5):
        token_list = tokenizer.texts_to_sequences([text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = np.argmax(model.predict(token_list), axis=-1)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        text += " " + output_word
    return text

seed_text = "cost of living"
print(predict_next_word(model, tokenizer, seed_text, max_sequence_len))


Epoch 1/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.0061 - loss: 5.1339
Epoch 2/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.0162 - loss: 5.0574
Epoch 3/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.0363 - loss: 4.9544
Epoch 4/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.0369 - loss: 4.8688
Epoch 5/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.0468 - loss: 4.8634
Epoch 6/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.0634 - loss: 4.7847
Epoch 7/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.0581 - loss: 4.7806
Epoch 8/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.0630 - loss: 4.8124
Epoch 9/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.0448 - loss: 4.8230
Epoch 10/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.0628 - loss: 4.7623
Epoch 11/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.0783 - loss: 4.7337
Epoch 12/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.0744 - lo